In [165]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook

In [153]:
os.getcwd()

'C:\\Users\\Maksym Shvets\\Documents\\SFolder\\Face_Type_Detection_Mobile_App\\TrainingModels'

In [144]:
def add_lacking_px(image, target_size=(168, 224)):
    """Add missing pixels to the edges so the image is of target_size.
    : param image: cv2 image
    : param target_size: target width and hegth of output image"""
    
    temp_im = (np.array(image) / 255).astype(np.float32)
    w, h = target_size

    def solve_px_partition(ax_size, target):
        if ax_size == target:
            return (0, 0)
        
        resid = target - ax_size % target
        
        if resid % 2 == 0:
            return resid // 2, resid // 2
        else:
            return resid // 2, resid // 2 + 1


    # Define margins to add to the edges.
    top, down = solve_px_partition(temp_im.shape[0], h)
    left, right = solve_px_partition(temp_im.shape[1], w)

    px_top = np.ones((top, temp_im.shape[1]))
    px_down = np.ones((down, temp_im.shape[1]))
    temp_im = np.vstack((px_top, temp_im, px_down))

    px_left = np.ones((temp_im.shape[0], left))
    px_right = np.ones((temp_im.shape[0], right))
    temp_im = np.column_stack((px_left, temp_im, px_right))

    
    return temp_im


In [173]:
for class_dir in os.listdir("../Data/testing_set")[:1]:
    os.makedirs(f"../Data/Processed/test/{class_dir}", exist_ok=True)
    
    for file in tqdm_notebook(os.listdir(f"../Data/testing_set/{class_dir}")):
        if not file.endswith(".jpg"):
            continue
            
        img = cv2.imread(f"../Data/testing_set/{class_dir}/{file}")
        try:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        except:
            continue
            
        height, width = img.shape[0], img.shape[1]
        while height > 224 or width > 168:
            height, width = int(height * 0.5), int(width * 0.5)

        img = cv2.resize(img, (width, height))
        # print(file, "\n", img.shape)

        # ADAPTIVE THRESHOLDING
        img = cv2.GaussianBlur(img, (3, 3), 0)
        thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 3, 3) 

        # ADD LACKING PIXELS TO THE EDGES
        processed = add_lacking_px(thresh, target_size=(168, 224))

#         plt.figure(figsize=(10, 6))
#         plt.imshow(processed, 'gray')
#         plt.show()
#         print("processed image shape:", processed.shape, "\n---------------------")

        cv2.imwrite(f"../Data/Processed/test/{class_dir}/{file}", processed * 255, [int(cv2.IMWRITE_PNG_COMPRESSION),0])
    

  0%|          | 0/200 [00:00<?, ?it/s]